In [1]:
import os, pickle
import numpy as np
import tensorflow as tf
from keras.models import *
from keras.optimizers.optimizer_experimental.adamw import AdamW

from settings import *
import data
from generator import *
from model import *
from callbacks import *
from utils import *
from tokenizer import *
import pretokenizer as mypretk

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:

	try:
		tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
		print('Using GPU :)')

	except RuntimeError as e:
		print(e)

else:
	print('Using CPU :(')

Using GPU :)


In [3]:
dataset = data.parse_dataset(DATASET_PATH)

print('Dataset size:', '{:,.0f}'.format(len(dataset)))

print('\n' + dataset[:500])

Importing parsed dataset...
Dataset size: 461,407,584

Ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !<nl>J'ai pas pu aller plus loin que la page 3, même si c'était intéressant !<eom>Moi-même, j'ai dû me forcer pour arriver jusqu'à la fin. Et pourtant ça ne devient pas moins intéressant. Ca fout les jetons... :)<eod>Service qui sera rendu au contribuable pour la modique some de 70 Millions par an.<eom>Eeeeh oui ! 70 millions pour une loi qui aura pour effet de .<eod>Est-ce qu'elle a vraiment commis des 


In [4]:
words = mypretk.split(dataset[:500])

print(dataset[:500].replace(' ', '_'))
print_tokens(words)

Ironie_:_l'article_disant_qu'on_est_plus_capable_de_se_concentrer_plus_de_2_pages,_en_fait_6_!<nl>J'ai_pas_pu_aller_plus_loin_que_la_page_3,_même_si_c'était_intéressant_!<eom>Moi-même,_j'ai_dû_me_forcer_pour_arriver_jusqu'à_la_fin._Et_pourtant_ça_ne_devient_pas_moins_intéressant._Ca_fout_les_jetons..._:)<eod>Service_qui_sera_rendu_au_contribuable_pour_la_modique_some_de_70_Millions_par_an.<eom>Eeeeh_oui_!_70_millions_pour_une_loi_qui_aura_pour_effet_de_.<eod>Est-ce_qu'elle_a_vraiment_commis_des_
Ironie_:_l'article_disant_qu'on_est_plus_capable_de_se_concentrer_plus_de_2_pages,_en_fait_6_!<nl>J'ai_pas_pu_aller_plus_loin_que_la_page_3,_même_si_c'était_intéressant_!<eom>Moi-même,_j'ai_dû_me_forcer_pour_arriver_jusqu'à_la_fin._Et_pourtant_ça_ne_devient_pas_moins_intéressant._Ca_fout_les_jetons..._:)<eod>Service_qui_sera_rendu_au_contribuable_pour_la_modique_some_de_70_Millions_par_an.<eom>Eeeeh_oui_!_70_millions_pour_une_loi_qui_aura_pour_effet_de_.<eod>Est-ce_qu'elle_a_vraiment_commis_des

In [5]:
tokenizer = Tokenizer(dataset)
vocab = tokenizer.vocab

print('\nVocab size:', '{:,.0f}'.format(len(vocab)), '\n')

for v in vocab:
	print(f'[{v}]', end = ' ')

Loading vocab...

Vocab size: 19,946 

['] [.] [,] [ de] [<nl>] [ la] [ le] [ que] [ pas] [ les] [est] [ et] [ à] [ l] [<eom>] [ des] [ un] [-] [ d] [ en] [ c] [ qui] [s] [ pour] [ je] [ qu] [ une] [ a] [ est] [ du] [<eod>] [/] [ ne] [ ça] [ dans] [ ce] [ n] [ plus] [ sur] [ tu] [ mais] [ (] [e] [ ?] [ il] [ "] [ai] [ j] [ au] [>] [ on] [ fait] [ par] [ avec] [ se] [ sont] [ ou] [ si] [ tout] [il] [ bien] [)] [ même] [a] [ y] [ faire] ["] [...] [ comme] [ me] [C] [ :] [ s] [on] [un] [Je] [en] [r] [ quand] [ peut] [ suis] [_] [ aussi] [ peu] [?] [://] [ être] [es] [ ont] [ C] [ leur] [ te] [ !] [ son] [ t] [).] [ Je] [ très] [ gens] [ non] [y] [ ils] [ dire] [é] [nt] [ aux] [J] [ mon] [ m] [ moins] [une] [ rien] [ sans] [ donc] [ils] [er] [ France] [t] [ là] [ va] [ ] [%] [ cette] [ juste] [ faut] [ parce] [ contre] [ ton] [ moi] [ dit] [ Et] [*] [ tous] [ quoi] [:] [ Le] [ ces] [ été] [ vraiment] [Et] [ soit] [ temps] [ mal] [être] [ sa] [Le] [était] [le] [n] [ lui] [ cas] [ avoir] [ m

In [6]:
if os.path.exists(os.path.join(PROCESSED_DATA_DIR, 'tokens.npy')):
	tokens = np.load(os.path.join(PROCESSED_DATA_DIR, 'tokens.npy'))

else:
	tokens = tokenizer.encode(dataset, True)
	np.save(os.path.join(PROCESSED_DATA_DIR, 'tokens.npy'), tokens)

print('\nEncoded dataset:')

for token in tokens[:100]:
	print(token, end = ' ')

print('\n\nDecoded dataset:')

example = tokenizer.decode(tokens[:100], True, True)
print_tokens(example)


Encoded dataset:
519 2080 292 71 13 0 402 1330 25 0 73 28 37 1485 3 54 8140 37 3 247 3508 2 19 51 674 92 4 106 0 46 8 474 418 37 420 7 5 1474 329 2 62 57 20 0 146 790 92 14 798 17 479 2 47 0 46 1119 69 3129 23 1375 548 0 235 5 445 1 130 747 33 32 1102 8 109 790 1 464 1187 9 6059 422 67 352 30 10291 12970 21 368 2203 48 8550 23 5 2431 450 10229 3 2294 14095 422 52 

Decoded dataset:
Ironie_:_l'article_disant_qu'on_est_plus_capable_de_se_concentrer_plus_de_2_pages,_en_fait_6_!<nl>J'ai_pas_pu_aller_plus_loin_que_la_page_3,_même_si_c'était_intéressant_!<eom>Moi-même,_j'ai_dû_me_forcer_pour_arriver_jusqu'à_la_fin._Et_pourtant_ça_ne_devient_pas_moins_intéressant._Ca_fout_les_jetons..._:)<eod>Service_qui_sera_rendu_au_contribuable_pour_la_modique_some_de_70_Millions_par


In [7]:
train_indexes, val_indexes = data.split_dataset(tokens)

print('Train indexes:', '{:,.0f}'.format(len(train_indexes)))
print('Val indexes:  ', '{:,.0f}'.format(len(val_indexes)))

Train indexes: 110,669,414
Val indexes:   5,824,860


In [8]:
train_generator = BatchGenerator(tokens, train_indexes, size = STEP_PER_EPOCH)
x, y = train_generator.__getitem__(0)

for i in range(len(x[0])):
	print(x[0][i], end = (max(len(str(x[0][i])), len(str(y[0][i]))) - len(str(x[0][i])) + 1) * ' ' + '| ')

print()

for i in range(len(y[0])):
	print(y[0][i], end = (max(len(str(x[0][i])), len(str(y[0][i]))) - len(str(y[0][i])) + 1) * ' ' + '| ')

print('\n')
print_tokens(tokenizer.decode(x[0], True, True))
print_tokens(tokenizer.decode(y[0], True, True))

17 | 63 | 17  | 209 | 17   | 1374 | 17  | 219 | 17   | 9175 | 73 | 17   | 4842 | 389 | 17  | 754 | 17  | 308 | 17   | 1884 | 1496 | 17  | 596 | 17    | 14057 | 2156 | 81   | 2035 | 1741 | 4384 | 400 | 1   | 724 | 95 | 14   | 1285 | 1517 | 40 | 16    | 13024 | 3   | 840 | 763  | 4504 | 22  | 324 | 26  | 580 | 313 | 38 | 15   | 1210 | 3031 | 22 | 34 | 13 | 0    | 4355 | 847 | 21 | 88  | 506 | 451 | 2   | 120 | 11 | 16    | 15147 | 18 | 0  | 74  | 359 | 170 | 39   | 5691 | 1172 | 20 | 0  | 10  | 930  | 5514 | 1 | 4 | 4   | 244 | 164 | 35 | 36 | 0  | 10 | 8  | 99 | 37 | 68 | 57 | 39 | 91    | 12208 | 9962  | 12263 | 21  | 468 | 15   | 2845 | 23  | 418  | 2502 | 29   | 4550 | 5581 | 19   | 2333 | 156 | 331  | 6026 | 4182 | 7373 | 1  | 30 | 75 | 80 | 16 | 83   | 1127 | 17   | 9747 | 2  | 40 | 
63 | 17 | 209 | 17  | 1374 | 17   | 219 | 17  | 9175 | 73   | 17 | 4842 | 389  | 17  | 754 | 17  | 308 | 17  | 1884 | 1496 | 17   | 596 | 17  | 14057 | 2156  | 81   | 2035 | 1741 | 4384 | 400  | 1   | 

In [9]:
model = create_model(len(tokenizer.vocab))

model.compile(
	optimizer = AdamW(learning_rate = 0.0, weight_decay = WEIGHT_DECAY, beta_1 = BETA_1, beta_2 = BETA_2, clipnorm = CLIP_GRADIENTS),
	loss = 'sparse_categorical_crossentropy',
	metrics = ['accuracy']
)

model.summary()

Model: "gradient_accumulate_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None)]       0           []                               
                                                                                                  
 token_embedding (TokenEmbeddin  multiple            15318528    ['input[0][0]',                  
 g)                                                               'final_layer_norm[0][0]']       
                                                                                                  
 position_embedding (PositionEm  (None, None, 768)   98304       ['input[0][0]']                  
 bedding)                                                                                         
                                                                          

In [10]:
init_epoch = 0

if os.path.exists(os.path.join(OUTPUT_DIR, 'logs.pkl')) and os.path.exists(os.path.join(OUTPUT_DIR, 'model.h5')):

	logs = pickle.load(open(os.path.join(OUTPUT_DIR, 'logs.pkl'), 'rb'))
	init_epoch = logs['epochs'][-1]

	model.fit(BatchGenerator(tokens, train_indexes, size = NUM_ACCUMULATIONS + 1), batch_size = BATCH_SIZE, epochs = 1, shuffle = False, verbose = 0)
	model.load_weights(os.path.join(OUTPUT_DIR, 'model.h5'))

In [11]:
model.fit(
	train_generator,
	validation_data = BatchGenerator(tokens, val_indexes, size = VAL_STEPS),
	batch_size = BATCH_SIZE,
	validation_batch_size = BATCH_SIZE,
	epochs = NUM_EPOCHS,
	shuffle = False,
	initial_epoch = init_epoch,
	callbacks = [
		LRScheduler(),
		SaveModel(),
		SaveLogs()
	]
)

Epoch 1/10000
500/500 [==============================] - 100s 157ms/step - loss: 9.5310 - accuracy: 0.0291 - lr: 9.3562e-06 - val_loss: 8.9150 - val_accuracy: 0.0463
Epoch 2/10000
500/500 [==============================] - 67s 134ms/step - loss: 8.6759 - accuracy: 0.0465 - lr: 2.8106e-05 - val_loss: 8.4424 - val_accuracy: 0.0452
Epoch 3/10000
500/500 [==============================] - 65s 130ms/step - loss: 8.2247 - accuracy: 0.0464 - lr: 4.6856e-05 - val_loss: 7.9240 - val_accuracy: 0.0457
Epoch 4/10000
500/500 [==============================] - 64s 127ms/step - loss: 7.6161 - accuracy: 0.0614 - lr: 6.5606e-05 - val_loss: 7.2584 - val_accuracy: 0.0763
Epoch 5/10000
500/500 [==============================] - 64s 128ms/step - loss: 6.9857 - accuracy: 0.0942 - lr: 8.4356e-05 - val_loss: 6.7362 - val_accuracy: 0.1059
Epoch 6/10000
500/500 [==============================] - 64s 128ms/step - loss: 6.6326 - accuracy: 0.1105 - lr: 1.0311e-04 - val_loss: 6.5086 - val_accuracy: 0.1212
Epoch 7/1

KeyboardInterrupt: 

In [ ]:
model.load_weights('./output/best_model.h5')

In [ ]:
model.evaluate(BatchGenerator(tokens, val_indexes, size = STEP_PER_EPOCH), batch_size = BATCH_SIZE)

In [ ]:
input = "<eod>Je vais te tuer ici et maintenant.<eom>"
output = predict(model, input, tokenizer, max_length = 256, temperature = 0.5, top_p = 0.95, no_repeat = 1.0, verbose = True)